In [45]:
# Importing the required libraries

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime 

In [46]:
url = 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'

table_attribs = ['Country','GDP_USD_millions']

db_name = 'World_Economies.db'

table_name = 'Countries_by_GDP'

csv_path = 'Countries_by_GDP.csv'

In [47]:
def extract(url, table_attribs):
    ''' This function extracts the required
    information from the website and saves it to a dataframe. The
    function returns the dataframe for further processing. '''
    page = requests.get(url).text
    data = BeautifulSoup(page,'html.parser')
    df = pd.DataFrame(columns=table_attribs)
    tables = data.find_all('tbody')
    rows = tables[2].find_all('tr')
    for row in rows:
        col = row.find_all('td')
        if len(col)!=0:
            if col[0].find('a') is not None and '—' not in col[2]:
                data_dict = {"Country": col[0].a.contents[0],
                             "GDP_USD_millions": col[2].contents[0]}
                df1 = pd.DataFrame(data_dict, index=[0])
                df = pd.concat([df,df1], ignore_index=True)
    return df

In [48]:
def transform(df):
    ''' This function converts the GDP information from Currency
    format to float value, transforms the information of GDP from
    USD (Millions) to USD (Billions) rounding to 2 decimal places.
    The function returns the transformed dataframe.'''
    
    df = extract(url, table_attribs)
    df['GDP_USD_millions'] =  [float(x.replace(",","")) for x in df['GDP_USD_millions']]
    df['GDP_USD_millions'] =  [round(x,-1)/1000 for x in df['GDP_USD_millions']]
    df.rename(columns = {'GDP_USD_millions':'GDP_USD_billions'}, inplace = True)

    return df

In [ ]:
def load_to_csv(df, csv_path):
    ''' This function saves the final dataframe as a `CSV` file 
    in the provided path. Function returns nothing.'''
    
    df.to_csv(csv_path, index = False)

In [49]:
def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final dataframe as a database table
    with the provided name. Function returns nothing.'''
    
    conn = sqlite3.connect(sql_connection)
    df.to_sql(table_name,conn,if_exists='replace',index=False)

In [ ]:
def run_query(query_statement, sql_connection):
    ''' This function runs the stated query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

In [90]:
url = 'https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks'
table_attribs_input =  ['Name','MC_USD_Billion']
table_attribs_output = ['Name','MC_USD_Billion','MC_GBP_Billion','MC_EUR_Billion','MC_INR_Billion']
path_output = './Largest_banks_data.csv'
database = 'Banks.db'
table_name = 'Largest_banks'

page = requests.get(url).text
soup = BeautifulSoup(page, "html.parser")
df = pd.DataFrame(columns = table_attribs_input)

table = soup.find_all("tbody")[0]
rows = table.find_all('tr')
for row in rows:
    col = row.find_all("td")
    if len(col)!=0:
        name = col[1].find_all('a')[1]['title']
        amount = float(col[2].contents[0].replace("\n",''))
        
        data_dict = {'Name':name,
                     'MC_USD_Billion':amount}
        
        df1 =  pd.DataFrame(data_dict, index = [0])
        df = pd.concat([df,df1], ignore_index = True)

df


/var/folders/8p/wjrnxtq910g62m0r4449h6zr0000gn/T/ipykernel_866/1057787430.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,df1], ignore_index = True)


,Name,MC_USD_Billion
0,JPMorgan Chase,432.92
1,Bank of America,231.52
2,Industrial and Commercial Bank of China,194.56
3,Agricultural Bank of China,160.68
4,HDFC Bank,157.91
5,Wells Fargo,155.87
6,HSBC,148.90
7,Morgan Stanley,140.83
8,China Construction Bank,139.82
9,Bank of China,136.81


In [95]:
rates = pd.read_csv("data/exchange_rate.csv")
dict = rates.set_index('Currency').to_dict()['Rate']

df['MC_GBP_Billion'] = [ np.round(x * dict['GBP'],2) for x in df['MC_USD_Billion']]
df['MC_EUR_Billion'] = [ np.round(x * dict['EUR'],2) for x in df['MC_USD_Billion']]
df['MC_INR_Billion'] = [ np.round(x * dict['INR'],2) for x in df['MC_USD_Billion']]

df

,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,JPMorgan Chase,432.92,346.34,402.62,35910.71
1,Bank of America,231.52,185.22,215.31,19204.58
2,Industrial and Commercial Bank of China,194.56,155.65,180.94,16138.75
3,Agricultural Bank of China,160.68,128.54,149.43,13328.41
4,HDFC Bank,157.91,126.33,146.86,13098.63
5,Wells Fargo,155.87,124.70,144.96,12929.42
6,HSBC,148.90,119.12,138.48,12351.26
7,Morgan Stanley,140.83,112.66,130.97,11681.85
8,China Construction Bank,139.82,111.86,130.03,11598.07
9,Bank of China,136.81,109.45,127.23,11348.39
